### Lab 5 - Samuel Akinnusi

#### Question 1
Please create a Model in either ArcGIS Pro for the Bobcat habitat example we discussed in class and export your model as both a python script and a image (lab5.py and lab5.jpg - 15 points)

In [19]:
import arcpy
from arcpy.sa import *

arcpy.env.workspace = "T:\\Students\\sakinnus\\SuitabilityModeling\\Output.gdb\\"
arcpy.env.overwriteOutput = True

# Check out any necessary licenses.
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

# Model Environment settings
# Set Geoprocessing environments
arcpy.env.snapRaster = "T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Elevation"
arcpy.env.extent = "439952.113762345 200181.284694512 513122.113762345 253671.284694512"
arcpy.env.cellSize = "30"
arcpy.env.mask = "T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Elevation"

Streams = "T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Streams"
Elevation = arcpy.Raster("T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Elevation")
LandUse = arcpy.Raster("T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\LandUse")

# Process: Distance Accumulation (Distance Accumulation) (sa)
DistToStrm = "DistToStrm"
Distance_Accumulation = DistToStrm
blkRaster = "blkRaster"
osdr = "osdr"
oslr = "oslr"
DistToStrm = arcpy.sa.DistanceAccumulation(in_source_data=Streams, vertical_factor="BINARY 1 -30 30", horizontal_factor="BINARY 1 45", out_back_direction_raster=blkRaster, out_source_direction_raster=osdr, out_source_location_raster=oslr, distance_method="PLANAR")
DistToStrm.save(Distance_Accumulation)

blkRaster = arcpy.Raster(blkRaster)
osdr = arcpy.Raster(osdr)
oslr = arcpy.Raster(oslr)

# Process: Slope (Slope) (sa)
SlopeMNew = "SlopeMNew"
Slope = SlopeMNew
SlopeMNew = arcpy.sa.Slope(in_raster=Elevation, output_measurement="DEGREE", z_factor=0.30480060960121924, method="PLANAR", z_unit="Foot_US")
SlopeMNew.save(Slope)


# Process: Reclassify (Reclassify) (sa)
Rcl_slope = "Rcl_slope"
Reclassify = Rcl_slope
Rcl_slope = arcpy.sa.Reclassify(in_raster=SlopeMNew, reclass_field="VALUE", remap="0 3 1;3 10 3;10 25 6;25 90 10", missing_values="DATA")
Rcl_slope.save(Reclassify)


# Process: Reclassify (2) (Reclassify) (sa)
Rcl_landuse = "Rcl_landuse"
Reclassify_2_ = Rcl_landuse
Rcl_landuse = arcpy.sa.Reclassify(in_raster=LandUse, reclass_field="VALUE", remap="1 1;2 1;3 1;4 2;5 8;6 10;7 10;8 10;9 6;10 2;11 10;12 1", missing_values="DATA")
Rcl_landuse.save(Reclassify_2_)


# Process: Rescale by Function (Rescale by Function) (sa)
Resc_DistToStrm = "Resc_DistToStrm"
Rescale_by_Function = Resc_DistToStrm
Resc_DistToStrm = arcpy.sa.RescaleByFunction(in_raster=DistToStrm, transformation_function=[["MSSMALL", "", "", "", "", 1, 1, ""]], from_scale=1, to_scale=10)
Resc_DistToStrm.save(Rescale_by_Function)


# Process: Raster Calculator (Raster Calculator) (sa)
SIndexMap = "SIndexMap"
Raster_Calculator = SIndexMap
SIndexMap =  Rcl_slope *0.3 + Rcl_landuse *0.3 + Resc_DistToStrm *0.4
SIndexMap.save(Raster_Calculator)

#### Question 2
Modify the script (lab5.py) so it will allow a user to configure weights of slope, landuse, and distance to streams. (10 points)

In [25]:
import arcpy
from arcpy.sa import *

arcpy.env.workspace = "T:\\Students\\sakinnus\\SuitabilityModeling\\Output.gdb\\"
arcpy.env.overwriteOutput = True

# Check out any necessary licenses.
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

# Model Environment settings
# Set Geoprocessing environments
arcpy.env.snapRaster = "T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Elevation"
arcpy.env.extent = "439952.113762345 200181.284694512 513122.113762345 253671.284694512"
arcpy.env.cellSize = "30"
arcpy.env.mask = "T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Elevation"

Streams = "T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Streams"
Elevation = arcpy.Raster("T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\Elevation")
LandUse = arcpy.Raster("T:\\Students\\sakinnus\\SuitabilityModeling\\Data.gdb\\LandUse")

# Process: Distance Accumulation (Distance Accumulation) (sa)
DistToStrm = "DistToStrm"
Distance_Accumulation = DistToStrm
blkRaster = "blkRaster"
osdr = "osdr"
oslr = "oslr"
DistToStrm = arcpy.sa.DistanceAccumulation(in_source_data=Streams, vertical_factor="BINARY 1 -30 30", horizontal_factor="BINARY 1 45", out_back_direction_raster=blkRaster, out_source_direction_raster=osdr, out_source_location_raster=oslr, distance_method="PLANAR")
DistToStrm.save(Distance_Accumulation)

blkRaster = arcpy.Raster(blkRaster)
osdr = arcpy.Raster(osdr)
oslr = arcpy.Raster(oslr)

# Process: Slope (Slope) (sa)
SlopeMNew = "SlopeMNew"
Slope = SlopeMNew
SlopeMNew = arcpy.sa.Slope(in_raster=Elevation, output_measurement="DEGREE", z_factor=0.30480060960121924, method="PLANAR", z_unit="Foot_US")
SlopeMNew.save(Slope)


# Process: Reclassify (Reclassify) (sa)
Rcl_slope = "Rcl_slope"
Reclassify = Rcl_slope
Rcl_slope = arcpy.sa.Reclassify(in_raster=SlopeMNew, reclass_field="VALUE", remap="0 3 1;3 10 3;10 25 6;25 90 10", missing_values="DATA")
Rcl_slope.save(Reclassify)


# Process: Reclassify (2) (Reclassify) (sa)
Rcl_landuse = "Rcl_landuse"
Reclassify_2_ = Rcl_landuse
Rcl_landuse = arcpy.sa.Reclassify(in_raster=LandUse, reclass_field="VALUE", remap="1 1;2 1;3 1;4 2;5 8;6 10;7 10;8 10;9 6;10 2;11 10;12 1", missing_values="DATA")
Rcl_landuse.save(Reclassify_2_)


# Process: Rescale by Function (Rescale by Function) (sa)
Resc_DistToStrm = "Resc_DistToStrm"
Rescale_by_Function = Resc_DistToStrm
Resc_DistToStrm = arcpy.sa.RescaleByFunction(in_raster=DistToStrm, transformation_function=[["MSSMALL", "", "", "", "", 1, 1, ""]], from_scale=1, to_scale=10)
Resc_DistToStrm.save(Rescale_by_Function)


# Process: Raster Calculator (Raster Calculator) (sa)
SIndexMap = "SIndexMap"
Raster_Calculator = SIndexMap
userinput = input('Input the weight for the slope, landuse, and Distance to stream (e.g. 0.4 0.2 0.4):')
weight = userinput.split()
SIndexMap =  Rcl_slope *float(weight[0]) + Rcl_landuse *float(weight[1]) + Resc_DistToStrm *float(weight[2])
SIndexMap.save(Raster_Calculator)

Input the weight for the slope, landuse, and Distance to stream (e.g. 0.4 0.2 0.4):0.4 0.3 0.3
